In [12]:
#Import package
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib
from geopy.distance import geodesic
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
from datetime import datetime, timedelta, date
import pivottablejs as pvt
import math

ModuleNotFoundError: No module named 'pivottablejs'

In [8]:
# - Import data -
ambulances = pd.read_parquet('ambulance_locations.parquet.gzip')
postal_codes_subset = pd.read_csv('postal_codes.csv', sep=',')
# ERROR, why ?
postal_codes_subset.head()

,"postcode,smun_code,reg_code,prov_code,arr_code,mun_code,Region,Province,Arrondissement,Municipality"
0,"3211,24066B,02000,20001,24000,24066,Flanders,V..."
1,"1910,23038A,02000,20001,23000,23038,Flanders,V..."
2,"9120,46003A,02000,40000,46000,46003,Flanders,O..."
3,"9520,41063A,02000,40000,41000,41063,Flanders,O..."
4,"9667,45062B,02000,40000,45000,45062,Flanders,O..."


In [ ]:
# ambulances
ambulances.head()
ambulances_cl=ambulances.copy()

regions = {'Brussels Hoofdstedelijk Gewest':'Brussels','Vlaams Gewest':'Flanders','Waals Gewest':'Wallonia','null':'null'}
ambulances_cl['region']=ambulances_cl['region'].map(regions)

for column in ambulances_cl.columns:
    if ambulances_cl[column].dtype=='str':
        ambulances_cl[column]=ambulances_cl[column].str.title().str.strip()
    else:
        ambulances_cl[column]=ambulances_cl[column]      
  
for index, row in ambulances_cl.iterrows():
    item = row['departure_location']
    strings = item.split()
    ambulances_cl.at[index,'departure_municipality']=item.split()[-1].title()
    for string in strings:
        if (len(string) == 4) and (string.isdigit()):
            ambulances_cl.at[index, 'departure_postalcode'] = string    

ambulances_all = pd.merge(ambulances_cl,postal_codes_subset, left_on = 'departure_postalcode', right_on = 'postcode', how ='left')
for a, b in zip(ambulances_all.columns, range(0,len(ambulances_all.columns))):
                print(f'{a}, column number {b}')

ambulances_subset = ambulances_all.iloc[:,[0,2,3,4,5,6,7,9,10,17,18,19]]
ambulances_subset.head()

In [ ]:
ambulances_subset.to_csv('ambulances.csv', index=False)

In [2]:
ambulances_df = pd.read_excel('ambulancesV2.xlsx')

In [27]:
import csv
import string

input_file = open('ambulances.csv', 'r')
output_file = open('ambulances2.csv', 'w')
data = csv.reader(input_file)
writer = csv.writer(output_file)
specials = "a"
print(specials)

# for line in data:
#     line = str(line)
#     new_line = str.replace(line,specials,'')
#     writer.writerow(new_line.split(','))

for line in data:
    line = [value.replace(specials, '') for value in line]
    writer.writerow(line)

input_file.close()
output_file.close()

a


In [24]:
ambulances_df = pd.read_csv('ambulances.csv', sep=',')
#not working, why ?
ambulances_df.head()

,"base,province,region,departure_location,departure_location_number,latitude,longitude,departure_municipality,departure_postalcode,Region,Province,Arrondissement"
0,"BA ANTW [Borgerhout] AMBUCE,Antwerpen,Flanders..."
1,"BA ANTW [Hoboken] AMBUCE,Antwerpen,Flanders,An..."
2,"BA ANTW [Hof Ter Schelde] AC Antwerpen,Antwerp..."
3,"BA ANTW [Kaai 606] SIHWA,Antwerpen,Flanders,An..."
4,"BA ANTW [Sint Elisabeth] AC Antwerpen,Antwerpe..."


In [3]:
# Nearest Neighbour
# 1. Clean a bit more ambulance dataframe
ambulances_df.head(10)
amb_df = ambulances_df[['base','latitude','longitude','Arrondissement']]
# Create a Tuple 'coordonate' from latitude and longitude
amb_df['coordonate'] = list(zip(amb_df.latitude, amb_df.longitude))
amb_df['coordonate'].head(10)
amb_df.head(10)

from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

# 2. Create a list of coordonate (amb_list) 
# and a Mulitpoint of coordonate (departures)
# for Nearest Neighbour method
amb_list = amb_df['coordonate'].tolist() # TODO into function !!
departures= MultiPoint(amb_list)
# print(departures)

# Make to coordonate Tuple into Point
amb_df['PointCoordonate'] = amb_df.apply(lambda x: Point(x['latitude'], x['longitude']), axis=1)
print(amb_df['PointCoordonate'].head(10))

# Test
testpoint = Point(50.5641805, 4.4907038)
amb_df.loc[amb_df['PointCoordonate'] == testpoint]
# print((testpoint.x,testpoint.y))

# # EXAMPLE NN method
# orig = Point(50.5,4.5)
# nearest_depart = nearest_points(orig, departures)
# near_idx0 = nearest_depart[0]
# near_idx1 = nearest_depart[1]
# print(nearest_depart)
# print(near_idx0)
# print(near_idx1)

# Adding new ambulance to the list of departures
# departures.append([Point(newCoordonate)])

0    POINT (51.2155158 4.4437875)
1    POINT (51.1788314 4.3619482)
2     POINT (51.2328096 4.377237)
3    POINT (51.3155156 4.3215063)
4    POINT (51.2136226 4.4057177)
5      POINT (51.319819 5.089912)
6    POINT (51.3209834 4.8566561)
7    POINT (51.1140555 4.6472462)
8    POINT (51.4646726 4.4725507)
9     POINT (51.0430641 4.815907)
Name: PointCoordonate, dtype: object


C:\Users\Ramy\AppData\Local\Temp\ipykernel_21688\3066386475.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amb_df['coordonate'] = list(zip(amb_df.latitude, amb_df.longitude))
C:\Users\Ramy\AppData\Local\Temp\ipykernel_21688\3066386475.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amb_df['PointCoordonate'] = amb_df.apply(lambda x: Point(x['latitude'], x['longitude']), axis=1)


,base,latitude,longitude,Arrondissement,coordonate,PointCoordonate
277,BF SART BHN AMBU,50.56418,4.490704,Nivelles,"(50.5641805, 4.4907038)",POINT (50.5641805 4.4907038)


In [ ]:
# Nearest Neighbour
# 1. Clean a bit more ambulance dataframe
ambulances_df.head(10)
amb_df = ambulances_df[['base','latitude','longitude','Arrondissement']]
# Create a Tuple 'coordonate' from latitude and longitude
amb_df['coordonate'] = list(zip(amb_df.latitude, amb_df.longitude))
amb_df['coordonate'].head(10)
amb_df.head(10)

from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

# 2. Create a list of coordonate (amb_list) 
# and a Mulitpoint of coordonate (departures)
# for Nearest Neighbour method
amb_list = amb_df['coordonate'].tolist() # TODO into function !!
departures= MultiPoint(amb_list)
# print(departures)

# Make to coordonate Tuple into Point
amb_df['PointCoordonate'] = amb_df.apply(lambda x: Point(x['latitude'], x['longitude']), axis=1)
print(amb_df['PointCoordonate'].head(10))

# Test
testpoint = Point(50.5641805, 4.4907038)
amb_df.loc[amb_df['PointCoordonate'] == testpoint]
# print((testpoint.x,testpoint.y))

In [ ]:
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

def create_coorodnate_multipoint(data_columns):
    """
    Take a column of coordonate date (latitude,longitude) and return a Mulitpoint

    Input:
        data columns of coodonate (latitude,longitude)
    Output:
        Multipoint of coorodnates
    """
    data_list = data_columns.tolist()
    data_multipoint = MultiPoint(data_list)
    return data_multipoint

def coorodnate_column(dataframe, data_latitude, data_longitude):
    """
    Add a 'coordonate' column in the dataframe based on the its latitude and longitude columns

    Input :
        column of latitude data (exemple: dataframe.latitude)
        column of longitude data (exemple: dataframe.longitude)
    
    Ouput :
        None
    """
    dataframe['coordonate'] = list(zip(data_latitude, data_longitude))